In [28]:
import pandas as pd 
import numpy as np 
import re

import torch 
import torch.nn as nn

In [29]:
def load_fixed_dataframe(original = "data/train.csv", updated = "data/train_updates_20220929.csv", was_fixed = False):

    def fix_tm_ph(row, update_map):
        update_vals = update_map.get(row["seq_id"], None)
        if update_vals is not None:
            row["tm"] = update_vals["tm"] #processing thermochemical stability metric (Spearman Correlation Coefficient)
            row["pH"] = update_vals["pH"] #iterating through pH values and re-evaluating for precision
        return row
    
    df = pd.read_csv(original)
    updated_df = pd.read_csv(updated)
    seq_id_phtm = updated_df[~pd.isna(updated_df["pH"])].groupby("seq_id")[["pH", "tm"]].first().to_dict("index")

    bad_seqs = updated_df[pd.isna(updated_df["pH"])]["seq_id"].to_list()

    df = df[~df["seq_id"].isin(bad_seqs)].reset_index(drop = True)
    df = df.apply(lambda x : fix_tm_ph(x, seq_id_phtm), axis = 1)

    if was_fixed: df["was_fixed"] = df["seq_id"].isin(bad_seqs + list(seq_id_phtm.keys()))
    return df 

def clean_train_df(df):
    df = df.drop(["seq_id", "data_source"], axis = 1)
    df = df.loc[~df['pH'].isna()]
    df = df.loc[df['pH'] <= 14]
    df = df.loc[df['protein_sequence'].str.len() <= 221]
    df = df.reset_index(drop = True)
    return df

train_df = load_fixed_dataframe()

In [30]:
train_df = clean_train_df(train_df)

In [31]:
'''
Pre-processing: Mapping encoding values to specified Amino Acid Indicators
'''

pH_list = list(train_df['protein_sequence'].str.split('').explode('protein_sequence').unique())
pH_list.remove('')
pH_map = {pH: i + 1 for i, pH in enumerate(pH_list)}
pH_map[None] = 0
pH_map

def encode(df):
    sequences_df = pd.DataFrame(df['protein_sequence'].apply(list).tolist())
    sequences_df = sequences_df.replace(pH_map)
    df = df.join(sequences_df)
    df = df.drop(columns=['protein_sequence'])
    return df

train_df = encode(train_df)

In [32]:
X, y = train_df.drop(["tm"], axis = 1), train_df["tm"]
n_samples = train_df.shape[0]

In [33]:
features = torch.tensor((X[:n_samples].values), dtype = torch.long)
labels = torch.tensor(y.values.reshape(-1, 1), dtype = torch.float32)

In [34]:
dataset = torch.utils.data.TensorDataset(features, labels)
n_split = int(len(dataset) * 0.98)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [n_split, len(dataset) - n_split])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 64)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64)

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EnzymeStabilityRegressor(nn.Module):
    def __init__(self, input_channels):
        super().__init__()

        self.n_layers = 1
        self.embedding = nn.Embedding(input_channels - 1, 256)
        self.lstm1 = nn.LSTM(256, 128, self.n_layers, bidirectional = True, batch_first = True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 1)

        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        outputs = self.embedding(x)
        outputs, (_h, _c) = self.lstm1(outputs)
        outputs = self.fc1(outputs[:, -1, :])
        outputs = self.activation(outputs)
        outputs = self.dropout(outputs)
        outputs = self.fc2(outputs)
        return outputs

In [36]:
model = EnzymeStabilityRegressor(222)
model = model.to(device)
print(model)

EnzymeStabilityRegressor(
  (embedding): Embedding(221, 256)
  (lstm1): LSTM(256, 128, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [37]:
import tqdm

loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
train_loss_all = []

for epoch in tqdm.trange(100):
    train_loss = 0
    train_num = 0
    
    for step,(X, y) in enumerate(train_dataloader):
        output = model(X)

        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * X.size(0)
        train_num += X.size(0)

    train_loss_all.append(train_loss / train_num)

  1%|          | 1/100 [03:35<5:54:51, 215.07s/it]


KeyboardInterrupt: 

In [ ]:
val_loss_all = []

val_loss = 0
val_num = 0

for step,(X, y) in enumerate(val_dataloader):
    X, y = X.to(device), y.to(device)

    output = model(X)

    loss = loss_fn(output, y)

    val_loss += loss.item() * X.size(0)
    val_num += X.size(0)

val_loss_all.append(val_loss / val_num)
print('val mse: ', sum(val_loss_all) / len(val_loss_all))

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
import os

model = EnzymeStabilityRegressor(222)
PATH = "C:/Users/kzhan/Desktop/thermoPySeqBio/state_dict"
torch.save(model.state_dict(), os.path.join(PATH, "lstmEmbed.pth"))
model.load_state_dict(torch.load(os.path.join(PATH, "lstmEmbed.pth")))

In [ ]:
test_df = pd.read_csv("data/test.csv")
test_df = clean_train_df(test_df)
test_df = encode(test_df)

test_dataset = torch.tensor((test_df.values), dtype=torch.long)

In [ ]:
output = model(test_dataset.to(device)).to(device)
result = output.cpu().data.numpy()
df_result = pd.DataFrame(result, columns=['tm'])

In [ ]:
test_df_1 = pd.read_csv("data/test.csv")

pred_df = pd.DataFrame({"pH": test_df["pH"], "Thermostability Coefficient": df_result["tm"],
"Protein Sequence": test_df_1["protein_sequence"]})

In [ ]:
pred_df